# Import necessary libraries

In [ ]:
#importing important libraries
!pip install pmdarima
import pandas as pd
import numpy as np
from pmdarima.arima import auto_arima
from pmdarima.arima import ARIMA

     |████████████████████████████████| 1.5MB 2.7MB/s 
     |████████████████████████████████| 2.1MB 17.9MB/s 
     |████████████████████████████████| 9.5MB 34.9MB/s 
  Found existing installation: Cython 0.29.21
    Uninstalling Cython-0.29.21:
      Successfully uninstalled Cython-0.29.21
  Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


# Necessary Functions

In [ ]:
# Mean Absolute Percentage Error
def mean_absolute_percentage_error(y_true, y_pred): 
  return np.round(np.mean(np.abs((y_pred - y_true) /np.abs(y_true))) * 100,decimals=2)
# Mean Absolute Error
def mean_absolute_error(y_true, y_pred): 
  return np.round(np.mean(np.abs((y_pred - y_true))),decimals=2)
# Mean Sqaured Error
def mean_squared_error( y_pred,y_true): 
  return np.round(np.mean((y_pred - y_true)**2),decimals=2)
# Root Mean Sqaured Error
def root_mean_squared_error(y_pred,y_true): 
  return np.round(np.sqrt(mean_squared_error(y_pred, y_true)),decimals=2)

# Read the data



1.   Read the input hospital data and then set the index to the 'Date' 
column.
2.   Aggregate the 3 German States Holiday columns for both School and Public Holidays
3. Create  seperate dataframes having Occupancy,Admissions and Discharges data to train the model




In [ ]:
#loading and conveting time series data by setting index as date
df = pd.read_csv('ProcessedDataset.csv')
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')
df.index = df.Date
#School holidays Aggregation
holidays_S = pd.DataFrame({'School_holiday': df[["S_BW", "S_H","S_RP"]].max(axis=1)})
holidays_S = holidays_S[["School_holiday"]]
holidays_P = pd.DataFrame({'Public_holiday': df[["P_BW", "P_H","P_RP"]].max(axis=1)})
holidays = holidays_P[["Public_holiday"]]
holidays['School_holiday'] = holidays_S.School_holiday

#Consider only the Occupancy
df = df.drop('Date', axis=1)
df_O = df[['Occupancy']]
df_D = df[['Discharges']]
df_A = df[['Admissions']]

# Train Test Split

In [ ]:
#divide into train and validation set
train_O = df_O['20080101':'20121231']
test_O = df_O['20130101':'20130130']
#Discharges
train_D = df_D['20080101':'20121231']
test_D = df_D['20130101':'20130130']
#Admissions 
train_A = df_A['20080101':'20121231']
test_A = df_A['20130101':'20130130']
#Holidays
holidays_train = holidays['20080101':'20121231']
holidays_test = holidays['20130101':'20130130']

# Discharges Model

## AUTO ARIMA Implementation
Running Auto Arima Model to find the p, d and q parameters values and predicting on the test data
Exogenous variables - School and Public Holidays of 3 German states

Best Model considered with Least AIC

In [ ]:
#training model
model_D = auto_arima(train_D, trace=True,start_p=0, start_q=0, start_P=0, start_Q=0,
                  max_p=10, max_q=10, max_P=10, max_Q=10, seasonal=True,
                  stepwise=True, suppress_warnings=True, D=1, max_D=10,
                  error_action='ignore',approximation = False,exogenous=holidays_train[['School_holiday','Public_holiday']])
#fitting model
model_D.fit(train_D)
#predicting values for test 
y_pred_D = model_D.predict(n_periods=test_D.shape[0],exogenous=holidays_test[['School_holiday','Public_holiday']])

In [ ]:
model_D

In [ ]:
predict for train
in_sample_preds_D = model_D.predict_in_sample()

#### Error Metrics
Calculation of MAPE, MAE and RMSE - Test

In [ ]:
#evaluating model
print('Mean absolute percentage error: ',mean_absolute_percentage_error(test_D.values, y_pred_D))
print('Mean absolute error: ',mean_absolute_error(test_D.values, y_pred_D))
print('Root Mean Square error: ',root_mean_squared_error(y_pred_D,test_D.values))

NameError: ignored

Calculation of MAPE, MAE and RMSE - Train

In [ ]:
print('Mean absolute percentage error: ',mean_absolute_percentage_error(train_D.values, in_sample_preds_D))
print('Mean absolute error: ',mean_absolute_error(train_D.values, in_sample_preds_D))
print('Root Mean Square error: ',root_mean_squared_error(in_sample_preds_D,train_D.values))

## ARIMA 
Run the model on the Identified ARIMA model with least AIC

### Implementation

In [ ]:
ar_D = ARIMA(maxiter=50, method='lbfgs', order=(4, 1, 1), out_of_sample_size=0,
      scoring='mse', scoring_args={}, seasonal_order=(0, 0, 0, 1),
      start_params=None, suppress_warnings=True, trend=None,
      with_intercept=True,exogenous=holidays_train[['School_holiday','Public_holiday']])
#fitting model
ar_D.fit(train_D)
#predicting values for test
y_pred_ar_D = ar_D.predict(n_periods=test_D.shape[0],exogenous=holidays_test[['School_holiday','Public_holiday']])

In [ ]:
#predict for train
in_sample_preds_ar_D = ar_D.predict_in_sample()

### Error Metrics
Calculation of MAPE, MAE and RMSE - Test

In [ ]:
#evaluating model
print('Mean absolute percentage error: ',mean_absolute_percentage_error(test_D.values, y_pred_ar_D))
print('Mean absolute error: ',mean_absolute_error(test_D.values, y_pred_ar_D))
print('Root Mean Square error: ',root_mean_squared_error(y_pred_ar_D,test_D.values))

Calculation of MAPE, MAE and RMSE - Train

In [ ]:
print('Mean absolute percentage error: ',mean_absolute_percentage_error(train_D.values, in_sample_preds_ar_D))
print('Mean absolute error: ',mean_absolute_error(train_D.values, in_sample_preds_ar_D))
print('Root Mean Square error: ',root_mean_squared_error(in_sample_preds_ar_D,train_D.values))

# Admissions Model

## AUTO ARIMA Implementation
Running Auto Arima Model to find the p, d and q parameters values and predicting on the test data
Exogenous variables - School and Public Holidays of 3 German states

Best Model considered with Least AIC

In [ ]:
training model
model_A = auto_arima(train_A, trace=True,start_p=0, start_q=0, start_P=0, start_Q=0,
                  max_p=10, max_q=10, max_P=10, max_Q=10, seasonal=True,
                  stepwise=False, suppress_warnings=True, D=1, max_D=10,
                  error_action='ignore',approximation = False,exogenous=holidays_train[['School_holiday','Public_holiday']])
#fitting model
model_A.fit(train_A)
#predicting values for test 
y_pred_A = model_A.predict(n_periods=test_A.shape[0],exogenous=holidays_test[['School_holiday','Public_holiday']])

In [ ]:
model_A

In [ ]:
#predict for train
in_sample_preds_A = model_A.predict_in_sample()

### Error Metrics
Calculation of MAPE, MAE and RMSE - Test

In [ ]:
# #evaluating model
# print('Mean absolute percentage error: ',mean_absolute_percentage_error(test_A.values, y_pred_A))
# print('Mean absolute error: ',mean_absolute_error(test_A.values, y_pred_A))
# print('Root Mean Square error: ',root_mean_squared_error(y_pred_A,test_A.values))

Calculation of MAPE, MAE and RMSE - Train

In [ ]:
# print('Mean absolute percentage error: ',mean_absolute_percentage_error(train_A.values, in_sample_preds_A))
# print('Mean absolute error: ',mean_absolute_error(train_A.values, in_sample_preds_A))
# print('Root Mean Square error: ',root_mean_squared_error(in_sample_preds_A,train_A.values))

## ARIMA 
Run the model on the Identified ARIMA model with least AIC

### Implementation

In [ ]:
ar_A = ARIMA(maxiter=50, method='lbfgs', order=(4, 1, 1), out_of_sample_size=0,
      scoring='mse', scoring_args={}, seasonal_order=(0, 0, 0, 1),
      start_params=None, suppress_warnings=True, trend=None,
      with_intercept=True,exogenous=holidays_train[['School_holiday','Public_holiday']])
#fitting model
ar_A.fit(train_A)
#predicting values for test
y_pred_ar_A = ar_A.predict(n_periods=test_A.shape[0],exogenous=holidays_test[['School_holiday','Public_holiday']])

In [ ]:
#predict for train
in_sample_preds_ar_A = ar_A.predict_in_sample()

### Error Metrics
Calculation of MAPE, MAE and RMSE - Test

In [ ]:
#evaluating model
print('Mean absolute percentage error: ',mean_absolute_percentage_error(test_A.values, y_pred_ar_A))
print('Mean absolute error: ',mean_absolute_error(test_A.values, y_pred_ar_A))
print('Root Mean Square error: ',root_mean_squared_error(y_pred_ar_A,test_A.values))

Calculation of MAPE, MAE and RMSE - Train

In [ ]:
print('Mean absolute percentage error: ',mean_absolute_percentage_error(train_A.values, in_sample_preds_ar_A))
print('Mean absolute error: ',mean_absolute_error(train_A.values, in_sample_preds_ar_A))
print('Root Mean Square error: ',root_mean_squared_error(in_sample_preds_ar_A,train_A.values))

# Finding Occupancy from predicted Admisisons + Discharges 

Considering the previous occupancy of the last day of training data to find the Occupancy based on the predicted Admissions and Discharges

### Occupancy Calculation for the Test data 

In [ ]:
#Defining the Occupancy dataframe based on the predicted Admissions and Discharges
O_df = pd.DataFrame(columns = ['Date','Admissions','Discharges','Occupancy'])
O_df['Date'] = test_A.index
O_df['Admissions'] = y_pred_ar_A 
O_df['Discharges'] = y_pred_D
#Getting the Previous Occupancy of the last day of Training data
PreviousOccupancy = 114

#Empty dataframe to append the calculated Occupancy
Occupancy = []

#Loop to calculate Occupancy for the Test data based on the predicted Admissions and Discharges
for i,row in O_df.iterrows():
  PreviousOccupancy = PreviousOccupancy + row['Admissions'] - row['Discharges']
  Occupancy.append(PreviousOccupancy)

O_df['Occupancy'] = Occupancy

In [ ]:
O_df.to_csv(r'O_df_ARIMA.csv')

### Occupancy Calculation for the Train data 

In [ ]:
#Defining the Occupancy dataframe based on the train Admissions and Discharges
O_dft = pd.DataFrame(columns = ['Date','Admissions','Discharges','Occupancy'])
O_dft['Date'] = train_A.index
O_dft['Admissions'] = in_sample_preds_ar_A
O_dft['Discharges'] = in_sample_preds_ar_D
#Getting the Previous Occupancy of the last day of Training data
PreviousOccupancy = 139
#Empty dataframe to append the calculated Occupancy
Occupancy = []

#Loop to calculate Occupancy for the Test data based on the predicted Admissions and Discharges
for i,row in O_dft.iterrows():
  PreviousOccupancy = PreviousOccupancy + row['Admissions'] - row['Discharges']
  Occupancy.append(PreviousOccupancy)

O_dft['Occupancy'] = Occupancy

Actual vs Predicted Occupancy plot

## Error Metrics


MAPE,RMSE And MAE for Calculated Occupancy - Test

In [ ]:
print('Mean absolute percentage error: ',mean_absolute_percentage_error(test_O['Occupancy'].values,O_df['Occupancy'].values))
print('Mean absolute error: ',mean_absolute_error(test_O['Occupancy'].values,O_df['Occupancy'].values))
print('Root Mean Square error: ',root_mean_squared_error(O_df['Occupancy'].values,test_O['Occupancy'].values))

MAPE,RMSE And MAE for Calculated Occupancy - Train

In [ ]:
print('Mean absolute percentage error: ',mean_absolute_percentage_error(train_O['Occupancy'].values,O_dft['Occupancy'].values))
print('Mean absolute error: ',mean_absolute_error(train_O['Occupancy'].values,O_dft['Occupancy'].values))
print('Root Mean Square error: ',root_mean_squared_error(O_dft['Occupancy'].values,train_O['Occupancy'].values))

# Plots



Actual vs Predictions (Test)

In [ ]:
# A vs P plot
ax = O_df.plot(x='Date',y='Occupancy',label='Predictions',legend=True,figsize=(6,4))
test_O.plot(y='Occupancy',label='Actual',legend=True,ax=ax)

In [ ]:
O_df


Actual vs Predictions (Train)

In [ ]:
# A vs P plot
ax = O_dft.plot(x='Date',y='Occupancy',label='Predictions',legend=True,figsize=(6,4))
train_O.plot(y='Occupancy',label='Actual',legend=True,ax=ax)

# 12 months rolling evaluation

In [ ]:
# Initialisation model
Train_Start = pd.to_datetime('01-01-2008',format='%d-%m-%Y')
# replace year only
Train_End = pd.to_datetime('31-12-2012',format='%d-%m-%Y')
print(Train_Start)
print(Train_End)

Test_Start = Train_End + pd.to_timedelta(1, unit='D')
Test_End= Test_Start + pd.to_timedelta(29, unit='D')
print(Test_Start)
print(Test_End)

training_time = pd.DataFrame()
mape = []
mae = []
rmse = []
Train_S =[]
Train_E =[]
Test_S =[]
Test_E =[]

#only for occupancy calculation
Hospital_df = pd.read_csv('ProcessedDataset.csv',parse_dates=True)
# Change the date format
Hospital_df['Date'] = pd.to_datetime(Hospital_df['Date'], format='%d-%m-%Y')
Hospital_df.index = Hospital_df['Date']

for i in range(1,13):
  print('********************************')
  print(i)
  Train_S.append(Train_Start)
  Train_E.append(Train_End)
  Test_S.append(Test_Start)
  Test_E.append(Test_End)

  # Before Training
  Test_df_D = df_D.loc[Test_Start:Test_End]
  Train_df_D = df_D.loc[Train_Start:Train_End]
  Test_df_A = df_A.loc[Test_Start:Test_End]
  Train_df_A = df_A.loc[Train_Start:Train_End]
  Test_df_O = Hospital_df.loc[Test_Start:Test_End]
  Train_df_O = Hospital_df.loc[Train_Start:Train_End]
  #arima discharges model
  ar_D = ARIMA(maxiter=50, method='lbfgs', order=(4, 1, 1), out_of_sample_size=0,
      scoring='mse', scoring_args={}, seasonal_order=(0, 0, 0, 1),
      start_params=None, suppress_warnings=True, trend=None,
      with_intercept=True,exogenous=holidays_train[['School_holiday','Public_holiday']])
  #fitting model
  ar_D.fit(Train_df_D)
  #predicting values for test
  y_pred_ar_D = ar_D.predict(n_periods=Test_df_D.shape[0],exogenous=holidays_test[['School_holiday','Public_holiday']])

  #arima admissions model
  ar_A = ARIMA(maxiter=50, method='lbfgs', order=(4, 1, 1), out_of_sample_size=0,
      scoring='mse', scoring_args={}, seasonal_order=(0, 0, 0, 1),
      start_params=None, suppress_warnings=True, trend=None,
      with_intercept=True,exogenous=holidays_train[['School_holiday','Public_holiday']])
  #fitting model
  ar_A.fit(Train_df_A)
  #predicting values for test
  y_pred_ar_A = ar_A.predict(n_periods=Test_df_A.shape[0],exogenous=holidays_test[['School_holiday','Public_holiday']])

  #Defining the Occupancy dataframe based on the predicted Admissions and Discharges
  O_df = pd.DataFrame(columns = ['Date','Admissions','Discharges','Occupancy'])
  O_df['Date'] = test_A.index
  O_df['Admissions'] = y_pred_ar_A 
  O_df['Discharges'] = y_pred_ar_D
  #Getting the Previous Occupancy of the last day of Training data
  PreviousOccupancy = Train_df_O.iloc[-1, 3]
  print(PreviousOccupancy)
  #Empty dataframe to append the calculated Occupancy
  Occupancy = []

  #Loop to calculate Occupancy for the Test data based on the predicted Admissions and Discharges
  for i,row in O_df.iterrows():
    PreviousOccupancy = PreviousOccupancy + row['Admissions'] - row['Discharges']
    Occupancy.append(PreviousOccupancy)

  O_df['Occupancy'] = Occupancy
  mape_eachtraining = mean_absolute_percentage_error(Test_df_O['Occupancy'].values,O_df['Occupancy'].values)
  rmse_eachtraining = root_mean_squared_error(O_df['Occupancy'].values,Test_df_O['Occupancy'].values)
  mae_eachtraining = mean_absolute_error(Test_df_O['Occupancy'].values,O_df['Occupancy'].values)
  print('Mean absolute percentage error: ',mape_eachtraining)
  mape.append(mape_eachtraining)
  rmse.append(rmse_eachtraining)
  mae.append(mae_eachtraining)

  # After Training
  Train_Start = Train_Start + pd.to_timedelta(30, unit='D')
  Train_End = Train_End + pd.to_timedelta(30, unit='D')
  print(Train_Start)
  print(Train_End)

  Test_Start = Test_Start + pd.to_timedelta(30, unit='D')
  Test_End= Test_End + pd.to_timedelta(30, unit='D')
  print(Test_Start)
  print(Test_End)

training_time['Train_Start']= Train_S
training_time['Train_End']= Train_E
training_time['Test_Start']= Test_S
training_time['Test_End']= Test_E
training_time['MAPE']= mape
training_time['RMSE']= rmse
training_time['MAE']= mae

In [ ]:
print('Average MAPE on 12 Validation Sets:',np.mean(training_time['MAPE']))
training_time.to_csv('Model3.csv')

In [ ]:
training_time

In [ ]:
training_time